In [2]:
import os
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset, DatasetDict, concatenate_datasets
from transformers import BertModel, BertTokenizerFast, AutoModelForSequenceClassification

import evaluate 
from evaluate import evaluator

# from transformers import pipeline
from optimum.pipelines import pipeline
from optimum.onnxruntime import ORTModelForFeatureExtraction

In [3]:
path_to_bert = '/home/ubuntu/partisan_bias_detection/best_models/finetune_bert/final_bert_lr_5e5'

In [11]:
# CWD = os.getcwd()
# DATASET_DIR = os.path.join(CWD, 'data')

# dataset = load_from_disk(DATASET_DIR)
# # dataset = dataset.remove_columns('input_ids')
# dataset = dataset.rename_column('label', 'labels')
# dataset.set_format('torch')

# print(dataset)

print('loading dataset...')
CWD = os.getcwd()
DATASET_DIR = os.path.join(CWD, 'data_limited_text')

dataset = load_from_disk(DATASET_DIR)
dataset.set_format('torch')

print('dataset loaded')
# print(dataset)

loading dataset...
dataset loaded


In [5]:
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

# def limit(article):
#     article['text'] = tokenizer.decode(tokenizer.encode(article['text'], max_length=512, truncation=True), skip_special_tokens=True)
#     return article

# dataset['train'] = dataset['train'].map(limit, writer_batch_size=100)
# dataset['val'] = dataset['val'].map(limit, writer_batch_size=100)
# dataset['test'] = dataset['test'].map(limit, writer_batch_size=100)

# ds = DatasetDict(
#         train=dataset['train'],
#         val=dataset['val'],
#         test=dataset['test'],
#     )

# save_to = 'data_limited_text'
# ds.save_to_disk(save_to) 

In [12]:
print('loading model + pipeline...')
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True, padding=True)
# model = BertModel.from_pretrained(path_to_bert,output_hidden_states=True)
model = ORTModelForFeatureExtraction.from_pretrained(path_to_bert,provider="CUDAExecutionProvider", from_transformers=True, use_io_binding=False)
extractor = pipeline(model=model, tokenizer=tokenizer, task="feature-extraction", device=device, accelerator="ort")

print('model and pipeline loaded')

loading model + pipeline...


The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
Some weights of the model checkpoint at /home/ubuntu/partisan_bias_detection/best_models/finetune_bert/final_bert_lr_5e5 were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-03-15 04:52:19.421268715 [W:onnxruntime:, session_state.cc:1136 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT

model and pipeline loaded


2023-03-15 04:52:22.206584602 [W:onnxruntime:, session_state.cc:1136 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-03-15 04:52:22.206610522 [W:onnxruntime:, session_state.cc:1138 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [ ]:
torch.cuda.empty_cache()

data = dataset['train'].select(range(10000))

embeds = extractor(data['text'], batch_size=16)

In [ ]:
dataset_embeds = {}

print('generating embeddings for...')

for split in ['val', 'test']: #'train', 
    print(split)
    embeds = extractor(dataset[split]['text'], batch_size=16)
    dataset_embeds[split] = Dataset.from_dict({'input_embeds': embeds, 'labels': dataset[split]['labels']})

generating embeddings for...
val


In [ ]:
ds = DatasetDict(
        train=dataset_embeds['train'],
        val=dataset_embeds['val'],
        test=dataset_embeds['test'],
    )

In [ ]:
print('saving data...')
save_to = 'dataset_embeds'
ds.save_to_disk(save_to) 
print('data saved')